In [1]:
import numpy as np
from itertools import permutations

In [2]:
def load_corpus(path, num_lines):
    file = open(path, "r")

    one_grams = {}
    two_grams = {}
    three_gram = {}
    number_of_words = 0
    num_line_iter = 0 
    
    for line in file:
        
        line = line.lower().split(" ")

        if len(line) < 3: continue

        number_of_words += len(line)
        line_iter = 0

        while line_iter < len(line) - 2:

            word_1 = line[line_iter]
            word_2 = line[line_iter+1]
            word_3 = line[line_iter+2]

            if word_3[-1] == '\n': word_3 = word_3[:-1] 

            key_1 = word_1
            key_2 = (word_1, word_2)
            key_3 = (word_1, word_2, word_3)

            if key_1 in one_grams.keys():
                one_grams[key_1] += 1
            else:
                one_grams[key_1] = 1

            if key_2 in two_grams.keys():
                two_grams[key_2] += 1
            else:
                two_grams[key_2] = 1

            if key_3 in three_gram.keys():
                three_gram[key_3] += 1
            else:
                three_gram[key_3] = 1

            line_iter += 1

        key_2 = (word_2, word_3)
        if key_2 in two_grams.keys():
            two_grams[key_2] += 1
        else:
            two_grams[key_2] = 1


        key_1 = word_2
        if key_1 in one_grams.keys():
             one_grams[key_1] += 1
        else:
            one_grams[key_1] = 1

        key_1 = word_3
        if key_1 in one_grams.keys():
            one_grams[key_1] += 1
        else:
            one_grams[key_1] = 1


        num_line_iter += 1
        if num_line_iter == num_lines: break

    return three_gram, two_grams, one_grams,  number_of_words

In [3]:
num_lines = 4500000

three_grams, two_grams, one_grams, number_of_words = load_corpus('/content/polish_corpora.txt', num_lines)

In [4]:
def train_three_grams(path, start_line, num_lines):
    file = open(path, "r")

    three_gram = {}
    num_line_iter = 0

    for line in file:
        if num_line_iter < start_line:
            num_line_iter += 1
            continue
        
        line = line.lower().split(" ")

        if len(line) < 3: continue

        line_iter = 0

        while line_iter < len(line) - 2:

            word_1 = line[line_iter]
            word_2 = line[line_iter+1]
            word_3 = line[line_iter+2]

            if word_3[-1] == '\n': word_3 = word_3[:-1] 

            key_3 = (word_1, word_2, word_3)

            if key_3 in three_gram.keys():
                three_gram[key_3] += 1
            else:
                three_gram[key_3] = 1
            
            line_iter += 1

        num_line_iter += 1
        if num_line_iter == start_line + num_lines: break

    return three_gram

In [5]:
train_corpus = train_three_grams('/content/polish_corpora.txt', num_lines, 100000)

In [6]:
def train(one_grams, two_grams, three_grams, train_three_grams, number_of_words):

    lambdas = np.array([0, 0, 0])

    for three_gram in train_three_grams.keys():

        p3 = 0
        p2 = 0
        p1 = 0

        if three_gram in three_grams:

            p3 = three_grams[three_gram]/two_grams[three_gram[:2]]

        if three_gram[1:] in two_grams:

            p2 = two_grams[three_gram[1:]]/one_grams[three_gram[1]]

        if three_gram[2] in one_grams:

            p1 = one_grams[three_gram[2]]/number_of_words
        
        results = (p1, p2, p3)

        f = lambda i: results[i]
        best = max(range(len(results)), key=f)

        lambdas[best] += train_three_grams[three_gram]

    

    lambdas = (lambdas-np.mean(lambdas)) / np.std(lambdas)
    e_x = np.exp(lambdas - np.max(lambdas))
    lambdas = e_x / e_x.sum()

    return lambdas
        

In [7]:
lb = train(one_grams, two_grams, three_grams, train_corpus, number_of_words)
del train_corpus

In [8]:
lb

array([0.06205478, 0.2194937 , 0.71845152])

In [9]:
def test_three_gram(one_grams, two_grams, three_grams, lambdas, three_gram):


    result = 0

    if three_gram in three_grams:

        result += lambdas[2] * three_grams[three_gram]/two_grams[three_gram[:2]]

    if three_gram[1:] in two_grams:

        result += lambdas[1] * two_grams[three_gram[1:]]/one_grams[three_gram[1]]

    if three_gram[2] in one_grams:

        result += lambdas[0] * one_grams[three_gram[2]]/number_of_words

    return result

In [10]:
def test_sentence(one_grams, two_grams, three_grams, lambdas, sentence):

    sen_iter = 0
    result = 0

    while sen_iter < len(sentence) - 2:

        word_1 = sentence[sen_iter]
        word_2 = sentence[sen_iter+1]
        word_3 = sentence[sen_iter+2]

        three_gram = (word_1, word_2, word_3)

        result += test_three_gram(one_grams, two_grams, three_grams, lambdas, three_gram)
        sen_iter +=1
    
    return result

In [11]:
f = open('/content/test.txt', 'r')

points = 0

for line in f:
    
    line = line.split()

    res = []
    res_values = []
    perms = list(permutations(line))
    for perm in list(perms):
        res.append([perm])
        res_values.append(test_sentence(one_grams, two_grams, three_grams, lb, perm))

    res = np.array(res)
    res_values = np.array(res_values)

    best = np.argsort(res_values)[::-1]
    points += 1/(np.where(best == 0)[0][0]+1)

    print("Best order: {}".format(" ".join(res[best[0]][0])))
    print("True order: {}".format(" ".join(line)))
    print("Points for sentence: {}".format(1/(np.where(best == 0)[0][0]+1)))
    print("\n")

print("All points: {}".format(points))

Best order: przedmiot przetwarzanie to nasz języka naturalnego
True order: nasz przedmiot to przetwarzanie języka naturalnego
Points for sentence: 0.004975124378109453


Best order: michaliszyn prowadzi jakub jedną grupę
True order: jedną grupę prowadzi jakub michaliszyn
Points for sentence: 0.02


Best order: rychlikowski wykładowcą jest paweł
True order: wykładowcą jest paweł rychlikowski
Points for sentence: 0.0625


Best order: pozostałe dwie grupy prowadzi również pan paweł
True order: pan paweł prowadzi również dwie pozostałe grupy
Points for sentence: 0.005952380952380952


Best order: zadań wykładu będą pojawiać się na stronie listy
True order: na stronie wykładu będą pojawiać się listy zadań
Points for sentence: 0.002583979328165375


Best order: pracowniowe ćwiczeniowe będą listy to i
True order: będą to listy ćwiczeniowe i pracowniowe
Points for sentence: 0.0017667844522968198


Best order: pracowniowe ćwiczeniowe będą listy to i
True order: będą to listy pracowniowe i ćwicz